<a href="https://colab.research.google.com/github/jgamel/learn_n_dev/blob/PySpark/PySpark_Intro_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark on Google Colab

Apache Spark is a lightning-fast framework used for data processing that performs super-fast processing tasks on large-scale data sets. It also can distribute data processing tasks across multiple devices, on its own, or in collaboration with other distributed computing tools.

PySpark is the interface that gives access to Spark using the Python programming language. PySpark is an API developed in python for spark programming and writing spark applications in Python style, although the underlying execution model is the same for all the API languages.
Colab by Google is an incredibly powerful tool that is based on Jupyter Notebook. Since it runs on the Google server, we don’t need to install anything in our system locally, be it Spark or any deep learning model.

In this notebook, we will see how we can run PySpark in a Google Colaboratory notebook. We will also perform some basic data exploratory tasks common to most data science problems.

Mount Google Drive

In [54]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [55]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

### Setting up PySpark in Colab

Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to download Java.

In [56]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Next, we will download and unzip Apache Spark with Hadoop 2.7 to install it.

Apache Spark Versions:
http://apache.osuosl.org/spark/

In [57]:
!wget -q http://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz 
!tar xf spark-3.2.1-bin-hadoop2.7.tgz

Now, it’s time to set the ‘environment’ path.

In [58]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop2.7/"

Then we need to import the ‘findspark’ library that will locate Spark on the system and import it as a regular library.

In [59]:
import findspark
findspark.init()

Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.

In [60]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [61]:
spark

### Loading data into PySpark

Spark has a variety of modules to read data of different formats. It also automatically determines the data type for each column, but it has to go over it once.

For this workbook, there is a sample JSON dataset from Github. You can download the file directly into Colab using the ‘wget’ command like this:

In [62]:
!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2022-03-30 22:39:42--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



Now read this file into a Spark dataframe using the read module.

In [63]:
df = spark.read.json("/tmp/sample_books.json")

### Exploratory Data Analysis with PySpark

Let’s check out its Schema:

Before doing any slice & dice of the dataset, we should first be aware what all columns it has and its data types.

In [64]:
df.printSchema()


root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



Show me some Samples:

In [65]:
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



How big is the Dataset:

In [66]:
df.count()

13

Select a few columns of interest:

In [67]:
df.select('title', 'price', 'year_written').show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



Filter the Dataset:

In [68]:
# Get books that are written after 1950 & cost greater than $10
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50, False)


+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [69]:
# Get books that have Harry Porter in their title
df_filtered.select("title", "year_written").filter("title LIKE '%Harry Potter%'").distinct().show(20, False)

+------------+------------+
|title       |year_written|
+------------+------------+
|Harry Potter|2000        |
+------------+------------+



Using Pyspark SQL functions:

In [71]:
from pyspark.sql.functions import max

# Find the costliest book
maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+

